# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

[2. Aufgabe](#aufgabe2)

3. Aufgabe

4. Aufgabe
5. Aufgabe

[6. Aufgabe](#aufgabe6)

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Einlesen der CSV Dateien, welche die Stromerzeugungsdaten und die Börsenstrompreise enthalten, als Dataframe df_hourly.

Nach dem ersten Zusammenführen haben wir festgestellt, dass die Datumsspalte je nach CSV-Datei unterschiedliche Namen für Sommerzeit und Winterzeit hat. Dies führt dazu, dass beim Zusammenführen zwei separate Spalten entstehen. Um dies zu vermeiden, prüfen wir bereits beim Einlesen der Dateien die Spaltennamen und führen sie zu einer einheitlichen Spalte „Datum“ im df_hourly zusammen


In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [ ]:
# Sources for reading csv files from one folder[1],[2]
path = './Daten/Strompreisdaten'
# List all files (.csv) in the given path
csv_files = glob.glob(os.path.join(path, "*csv"))

df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        for column in df_temp.columns:
            # Combine the date columns, by getting rid of the naming difference in csv source
            if 'Datum (MESZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MESZ)':'DateTime'})
            elif 'Datum (MEZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MEZ)':'DateTime'})
        df_list.append(df_temp)
    except Exception as err: 
        print("Fehler beim Einlesen des Files: ", err)
    
df_hourly = pd.concat(df_list)
df_hourly


,DateTime,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2021-08-16 00:00:00,16015.81,19041.64,68.19
1,2021-08-16 01:00:00,15629.83,19420.11,65.03
2,2021-08-16 02:00:00,15322.51,19738.75,62.08
3,2021-08-16 03:00:00,15026.53,20822.64,60.13
4,2021-08-16 04:00:00,14987.63,21578.78,60.82
...,...,...,...,...
163,2021-04-25 19:00:00,18362.77,18593.89,65.93
164,2021-04-25 20:00:00,19477.05,16565.88,67.88
165,2021-04-25 21:00:00,19214.45,17819.31,66.00
166,2021-04-25 22:00:00,18964.11,18205.01,64.98


b) Zur besseren anschließenden Analyse überführen wir die Datumsspalte (vorher dytpe: object) in ein Date-Time-Format. Überprüfung, ob nach der Konvertierung invalide Datumswerte existieren, was nicht der Fall ist.
Alle anderen Spalten haben den dtype float64 und werden so belassen.
Entfernung aller Datensätze, die nicht im Betrachtungszeitraum liegen (2020-2024). Durch Filterung über das Jahr werden 48 Einträge entfernt, da diese nicht in den Betrachtungszeitraum fallen.

In [ ]:
print(df_hourly.shape)
# Converting the column "DateTime" from object to DateTime format
# Invalid values are converted to NaT (Not a Time)
# Source: [3]
df_hourly['DateTime'] = pd.to_datetime(df_hourly['DateTime'], errors="coerce")
print(df_hourly.dtypes)
print(df_hourly['DateTime'].isna().any())


# Create column "Date" from column "DateTime" [4]
df_hourly = df_hourly[(df_hourly['DateTime'].dt.year >= 2020) & (df_hourly['DateTime'].dt.year <= 2024)]

# 48 rows deleted
print(df_hourly.shape) 



(43896, 4)
DateTime                             datetime64[ns]
Leistung nicht erneuerbar (MW)              float64
Leistung erneuerbar (MW)                    float64
Day Ahead Auktion Preis (EUR/MWh)           float64
dtype: object
False
(43848, 4)


c) Beurteilung der Datenqualität und notwenige Datenbereinigungsschritte. 
Qualität der Datumsdaten wurde durch Konvertierung schon überprüft. df_hourly enthält nur Werte für die Jahre 2020-2024. Dataframe enthält keine Null-Werte.

In [5]:
# Check for missing values
print(df_hourly.isnull().sum())
print(df_hourly.notnull().sum())



DateTime                             0
Leistung nicht erneuerbar (MW)       0
Leistung erneuerbar (MW)             0
Day Ahead Auktion Preis (EUR/MWh)    0
dtype: int64
DateTime                             43848
Leistung nicht erneuerbar (MW)       43848
Leistung erneuerbar (MW)             43848
Day Ahead Auktion Preis (EUR/MWh)    43848
dtype: int64


d) Erzeugung eines Dataframes df_daily, das für jeden Tag die aggregierten Werte der Leistung der erneuerbaren und nicht erneuerbaren Energiequellen, sowie den täglichen Mittelwert des Börsenpreis enthält. Diese Art der Gruppierung ist sinnvoll für die weitere Analyse, um Trends auf Tagesbasis zu analysieren.

Zur Gruppierung nach Tagen machen wir aus der DateTime nur noch das Datum, indem wir die Uhrzeit entfernen. Anschließend gruppieren wir die Daten nach Datum, wobei die Leistung nicht erneuerbarer und erneuererbarer Energie jeweils für den Tag aufsummiert wird. Zudem gruppieren wir nochmals nach dem Tag und berechnen für jeden Tag den mittleren Börsenpreis. Abschließend joinen wir zu einem gemeinsamen Dataframe df_daily. Zur klaren Lesbarkeit benennen wir die DateTime Spalte wieder in Datum zurück.

In [10]:
#df_hourly ['DateTime'] = df_hourly['DateTime'].dt.date

df_daily = df_hourly.groupby('DateTime')[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)' ]].sum()

df_mean = df_hourly.groupby('DateTime')['Day Ahead Auktion Preis (EUR/MWh)'].mean()

df_daily = pd.merge(df_daily, df_mean, on='DateTime', how='left').reset_index()

# Rename column DateTime in Date
df_daily = df_daily.rename(columns = {"DateTime": "Datum"})

df_daily


,Datum,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2020-01-01,468837.03,441093.57,34.964167
1,2020-01-02,505002.46,605620.76,38.869167
2,2020-01-03,427798.33,968577.88,26.111250
3,2020-01-04,411321.40,987041.46,20.322917
4,2020-01-05,508187.80,514534.00,35.552500
...,...,...,...,...
1822,2024-12-27,708574.01,268571.36,121.706250
1823,2024-12-28,700316.27,275162.43,122.638333
1824,2024-12-29,615408.50,547203.48,105.280417
1825,2024-12-30,607675.26,783569.49,78.155833


### Aufgabe 2 (Explorative Analyse der Stromerzeugungs- und Preisdaten)
<a id = "aufgabe2"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


a) An Welchen 10 Tagen im Betrachtungszeitraum wurden am meisten Strom aus erneuerbaren Energieträgern erzeugt?

In [11]:
ten_highest_days = df_daily.sort_values(by = 'Leistung erneuerbar (MW)', ascending=False).head(10)
ten_highest_days


,Datum,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
1497,2024-02-06,331773.90,1380463.41,45.292917
1496,2024-02-05,331039.23,1358926.97,35.840833
1484,2024-01-24,351983.99,1316391.11,33.122917
1450,2023-12-21,275886.68,1308032.94,25.280417
1451,2023-12-22,242307.14,1289984.86,22.655833
827,2022-04-07,396999.16,1274027.01,74.153333
778,2022-02-17,381238.33,1271433.30,68.637083
1731,2024-09-27,198813.32,1269933.18,14.757500
1458,2023-12-29,221981.80,1261365.77,6.359583
41,2020-02-11,351153.81,1257586.20,11.620000


b) An welchem Tag im Betrachtungszeitraum wurde der höchste Börsenstrompreis verzeichnet und wie hoch war er?
An welchem Tag im Betrachtungszeitraum wurde der niedrigste Börsenstrompreis verzeichnet und wie hoch war er?

### Aufgabe 6 (Analyse von Stromtarif-Angeboten für Endkunden)
<a id = "aufgabe6"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a)  Einlesen der JSON-Preisvergleichdaten in einem DataFrame df_cust


In [12]:
path = './Daten/Endkundenpreise/'

def extract_date_from_filepath(file):
    file = file.replace(path, "")
    # Concatenate the date out of fixed year 2024 and month/day from folder name
    return "2024-" + file[0:5]

def extract_data_from_json(file):
    assert file.endswith(".json"), "Keine json-Datei uebergeben"
    try:
        # Read and load the json file
        df_temp = pd.read_json(json_files[i])
        # Transpose the table: convert the rows to columns
        df_temp = df_temp.T
        # Index = Number of entry in the json file
        df_temp['Angebotsnummer'] = df_temp.index
        # Add date from filename as column
        df_temp['Datum'] = extract_date_from_filepath(file)
        
        return df_temp
    except:
        print("Datei konnte nicht gelesen werden.")


path = './Daten/Endkundenpreise/'
df_list = []
# https://www.tutorialspoint.com/python/os_listdir.htm
for folder in os.listdir(path):
    combined_path = os.path.join(path, folder, "*json")
    json_files = glob.glob(combined_path)

    for i in range(len(json_files)):
        df_temp = extract_data_from_json(json_files[i])
        df_list.append(df_temp)
        
df_cust = pd.concat(df_list)
df_cust


,Postleitzahl,Jahresverbrauch,Abschlagszahlung,Grundpreis,Arbeitspreis,Preisgarantie,Vertragslaufzeit,Verlängerung,Kündigungsfrist,Preis im 1. Jahr*,...,Preis im 1. Jahr,Sofortbonus,Arbeitspreisrabatt,Angebotsnummer,Datum,Blitzbonus,Grundpreisrabatt:,Abschlagsrabatt,Zusätzlicher Aktionsbonus,Winterprämie
0,95478,4.000 kWh,monatlich,"21,54 €/Monat (258,43 €/Jahr)","18,00 Cent/kWh",1 Monat Preisfixierung,1 Monat,1 Monat,1 Monat,"81,54 €/Monat",...,NaN,NaN,NaN,0,2024-04-27,NaN,NaN,NaN,NaN,NaN
1,95478,4.000 kWh,monatlich,"15,83 €/Monat (189,91 €/Jahr)","21,28 Cent/kWh",1 Monat Nettopreisgarantie,1 Monat,1 Monat,1 Monat,"82,47 €/Monat",...,NaN,NaN,NaN,1,2024-04-27,NaN,NaN,NaN,NaN,NaN
2,95478,4.000 kWh,monatlich,"16,83 €/Monat (201,92 €/Jahr)","20,02 Cent/kWh",1 Monat Nettopreisgarantie,1 Monat,2 Wochen,2 Wochen,"83,55 €/Monat",...,NaN,NaN,NaN,2,2024-04-27,NaN,NaN,NaN,NaN,NaN
3,95478,4.000 kWh,monatlich,"12,82 €/Monat (153,83 €/Jahr)","26,59 Cent/kWh",12 Monate Preisfixierung,12 Monate,1 Monat,1 Monat,NaN,...,"86,25 €/Monat",NaN,NaN,3,2024-04-27,NaN,NaN,NaN,NaN,NaN
4,95478,4.000 kWh,monatlich,"12,84 €/Monat (154,07 €/Jahr)","26,59 Cent/kWh",12 Monate Preisfixierung,12 Monate,1 Monat,1 Monat,NaN,...,"86,27 €/Monat",NaN,NaN,4,2024-04-27,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,92526,4.000 kWh,monatlich,"14,80 €/Monat (177,63 €/Jahr)","32,66 Cent/kWh",12 Monate Preisfixierung,12 Monate,1 Monat,1 Monat,NaN,...,"95,14 €/Monat",120 €,NaN,15,2024-08-02,NaN,NaN,NaN,NaN,NaN
16,92526,4.000 kWh,monatlich,"15,34 €/Monat (184,03 €/Jahr)","28,97 Cent/kWh",12 Monate Nettopreisgarantie,12 Monate,1 Monat,1 Monat,NaN,...,"95,15 €/Monat",NaN,NaN,16,2024-08-02,NaN,NaN,NaN,NaN,NaN
17,92526,4.000 kWh,monatlich,"14,33 €/Monat (171,96 €/Jahr)","30,26 Cent/kWh",Preisfixierung bis 30.09.2025,12 Monate,1 Monat,1 Monat,NaN,...,"95,20 €/Monat",38 €,"1,00 Ct./kWh",17,2024-08-02,NaN,NaN,NaN,NaN,NaN
18,92526,4.000 kWh,monatlich,"16,84 €/Monat (202,08 €/Jahr)","24,43 Cent/kWh",1 Monat Preisgarantie,1 Monat,2 Wochen,2 Wochen,"95,36 €/Monat",...,NaN,NaN,NaN,18,2024-08-02,35 €,NaN,NaN,NaN,NaN


### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[1] https://www.geeksforgeeks.org/how-to-read-all-csv-files-in-a-folder-in-pandas/

[2] https://statistikguru.de/python/python-auflisten-dateien-verzeichnis.html

[3] https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

[4] https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html#pandas.Series.dt.year

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)